**CNN model for cifar10 dataset, using a custom model (data in memory) with and without data augmentation**

**Load and Preprocess the CIFAR-10 Dataset**

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical


In [2]:
# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize the pixel values to be between 0 and 1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# One-hot encode the labels
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 19s 0us/step


**Define a Custom CNN Model**

In [3]:
def create_custom_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


**Train the Model Without Data Augmentation**

In [4]:
# Create the model
model_without_augmentation = create_custom_cnn()

# Train the model
history_without_augmentation = model_without_augmentation.fit(
    train_images, train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    batch_size=64
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.4459 - loss: 1.5752 - val_accuracy: 0.5787 - val_loss: 1.1723
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6552 - loss: 0.9874 - val_accuracy: 0.6315 - val_loss: 1.0451
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7170 - loss: 0.8116 - val_accuracy: 0.6494 - val_loss: 1.0190
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7546 - loss: 0.7037 - val_accuracy: 0.6867 - val_loss: 0.9107
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7876 - loss: 0.6097 - val_accuracy: 0.6501 - val_loss: 1.0735
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8150 - loss: 0.5314 - val_accuracy: 0.6899 - val_loss: 0.9242
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8326 - loss: 0.4806 - val_accuracy: 0.6745 - val_loss: 1.0534
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8539 - loss: 0.4241 - val_accuracy: 

**Train the Model With Data Augmentation**

In [5]:
# Create a data generator with data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Create the model
model_with_augmentation = create_custom_cnn()

# Train the model using the data generator
history_with_augmentation = model_with_augmentation.fit(
    datagen.flow(train_images, train_labels, batch_size=64),
    epochs=10,
    validation_data=(test_images, test_labels),
    steps_per_epoch=len(train_images) // 64
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


781/781 ━━━━━━━━━━━━━━━━━━━━ 46s 52ms/step - accuracy: 0.4041 - loss: 1.6814 - val_accuracy: 0.5518 - val_loss: 1.2587
Epoch 2/10
  1/781 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5312 - loss: 1.0895

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 689us/step - accuracy: 0.5312 - loss: 1.0895 - val_accuracy: 0.5306 - val_loss: 1.3254
Epoch 3/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 39s 50ms/step - accuracy: 0.5857 - loss: 1.1640 - val_accuracy: 0.6130 - val_loss: 1.1101
Epoch 4/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 832us/step - accuracy: 0.5156 - loss: 1.3375 - val_accuracy: 0.6143 - val_loss: 1.1041
Epoch 5/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 39s 48ms/step - accuracy: 0.6314 - loss: 1.0279 - val_accuracy: 0.6711 - val_loss: 0.9424
Epoch 6/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 931us/step - accuracy: 0.6406 - loss: 1.0797 - val_accuracy: 0.6693 - val_loss: 0.9477
Epoch 7/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step - accuracy: 0.6691 - loss: 0.9460 - val_accuracy: 0.6930 - val_loss: 0.8740
Epoch 8/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.6250 - loss: 1.0686 - val_accuracy: 0.6928 - val_loss: 0.8748
Epoch 9/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step - accuracy: 0.6920 - loss: 0.8879 - val_accurac

**Compare the Results**

In [6]:
# Evaluate both models on the test dataset
test_loss_without_aug, test_acc_without_aug = model_without_augmentation.evaluate(test_images, test_labels, verbose=2)
test_loss_with_aug, test_acc_with_aug = model_with_augmentation.evaluate(test_images, test_labels, verbose=2)

print(f"Test accuracy without data augmentation: {test_acc_without_aug:.4f}")
print(f"Test accuracy with data augmentation: {test_acc_with_aug:.4f}")


313/313 - 1s - 3ms/step - accuracy: 0.6880 - loss: 1.0493
313/313 - 1s - 2ms/step - accuracy: 0.6033 - loss: 1.2008
Test accuracy without data augmentation: 0.6880
Test accuracy with data augmentation: 0.6033
